# Clase 2 - Clasificación
![picture](https://drive.google.com/uc?id=1Ar6KpqlsLxcLlnOrXPx6QZejStP6bV9P)

## Pre requisitos

Se actualiza fastai descargando y ejecutando el script *colab*

In [0]:
!curl -s https://course.fast.ai/setup/colab | bash

Dar permiso para acceder a mi Drive

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Nos aseguramos que cualquier cambio en una librería es recargado automáticamente y que que cualquier gráfica o imagen se muestre aquí

In [0]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

## Curso 2 - Clasificación

Importamos el módulo de visión de la librería fastai

In [0]:
from fastai.vision import *

### Obtener una lista de URLs de las imágenes

Ir a [Google Images](http://images.google.com) y buscar las fotos de lo que queremos clasificar, en este caso será


*   Black bears
*   Grizzly bears
*   Teddy bears



![image1](https://github.com/maximofn/Curso-Fastai/blob/master/Imagenes/Google_images_01.PNG?raw=true)

Ir hasta abajo y pulsar el botón ```Mostrar más resultados``` para obtener más imágenes

![image2](https://github.com/maximofn/Curso-Fastai/blob/master/Imagenes/Google_images_02.PNG?raw=true)

Cuando se tenga el número de imágenes deseado pulsar las teclas <kbd>Ctrl</kbd> <kbd>+</kbd> <kbd>Shift</kbd> <kbd>+</kbd> <kbd>j</kbd> en Windows/Linux, o <kbd>Cmd</kbd> <kbd>+</kbd> <kbd>Opt</kbd> <kbd>+</kbd> <kbd>j</kbd> en macOS, para abrir una consola de JavaScript

![image3](https://github.com/maximofn/Curso-Fastai/blob/master/Imagenes/Google_images_03.PNG?raw=true)

Escribir el siguiente sccript en la consola


```javascript
urls=Array.from(document.querySelectorAll('.rg_i')).map(el=> el.hasAttribute('data-src')?el.getAttribute('data-src'):el.getAttribute('data-iurl'));
window.open('data:text/csv;charset=utf-8,' + escape(urls.join('\n')));
```



![image4](https://github.com/maximofn/Curso-Fastai/blob/master/Imagenes/Google_images_04.PNG?raw=true)

Al darle al intro se abrirá una ventana que nos preguntará cómo llamar al archivo, en los tres casos de búsqueda llamarlos


*   urls_black.csv
*   urls_grizzly.csv
*   urls_teddys.csv



### Copiar los CSVs a nuestro drive

Copiar los tres CSVs a nuestro Google Drive, a continuación escribir la ruta dónde están guardados los CSVs. Si el directorio tiene espacios poner antes de cada espacio una barra invertida ( \ )

In [0]:
CSVs_folder = '/content/drive/My\ Drive/Colab\ Notebooks/Curso\ fastai/02\ Clasificacion'

In [0]:
%cd /content/data/
!mkdir bears
%cd /content/

In [0]:
!cp {CSVs_folder}/urls_black.csv /content/data/bears
!cp {CSVs_folder}/urls_teddys.csv /content/data/bears
!cp {CSVs_folder}/urls_grizzly.csv /content/data/bears

### Descargar las imágenes

In [0]:
classes = ['teddys','grizzly','black']

#### Black bears

In [0]:
folder = 'black'
file = 'urls_black.csv'

In [0]:
path = Path('data/bears')
dest = path/folder
dest.mkdir(parents=True, exist_ok=True)

In [0]:
download_images(path/file, dest, max_pics=200)

In [0]:
!ls /content/data/bears/black/

#### Grizzly bears

In [0]:
folder = 'grizzly'
file = 'urls_grizzly.csv'

In [0]:
path = Path('data/bears')
dest = path/folder
dest.mkdir(parents=True, exist_ok=True)

In [0]:
download_images(path/file, dest, max_pics=200)

In [0]:
!ls /content/data/bears/grizzly/

#### Teddy bears

In [0]:
folder = 'teddys'
file = 'urls_teddys.csv'

In [0]:
path = Path('data/bears')
dest = path/folder
dest.mkdir(parents=True, exist_ok=True)

In [0]:
download_images(path/file, dest, max_pics=200)

In [0]:
!ls /content/data/bears/teddys/

### Comprobar imágenes

Vamos a verificar las imágenes para borrar las que no se puedan abrir

In [0]:
for c in classes:
    print(c)
    verify_images(path/c, delete=True, max_size=500)

### Construcción de los datos para la red neuronal

In [0]:
%cd /content

In [0]:
np.random.seed(42)
#data = ImageDataBunch.from_folder(path, train=".", valid_pct=0.2,
#        ds_tfms=get_transforms(), size=224, num_workers=16).normalize(imagenet_stats)
data = ImageDataBunch.from_folder(path, train=".", valid_pct=0.2,
        size=224, num_workers=16).normalize(imagenet_stats)

In [0]:
data.classes

In [0]:
data.show_batch(rows=3, figsize=(8,8))

In [0]:
data.classes, data.c, len(data.train_ds), len(data.valid_ds)

### Entrenamiento del modelo

In [0]:
learn = cnn_learner(data, models.resnet34, metrics=error_rate)

In [0]:
learn.fit_one_cycle(4)

In [0]:
learn.save('stage-1')

In [0]:
learn.lr_find()
learn.recorder.plot()

In [0]:
learn.unfreeze()
learn.fit_one_cycle(4, max_lr=slice(7e-5,1e-6))

In [0]:
learn.save('stage-2')